# TKT4108 - Structural Dynamics 2
## Homework 2

<img src="Shear Frame_HW2.JPG">

### For a building in Trondheim with behavior factor q = 2, shown above, compute the following earthquake responses assuming Eurocode 8 response spectrum and ground type D:

**- Maximum displacement of first floor**  
**- Maximum base shear**  
**- Maximum overturning moment**  

**Use both Eurocode 8 procedure and theoretical method**